In [1]:
import tensorflow as tf

In [10]:
# 定义数据流图dataflow
a = tf.constant(5, name='input_a')
b = tf.constant(3, name='input_b')
c = a+b
d = a*b
e = c+d

In [11]:
# 创建TensorFlow session对象
sess = tf.Session()
output = sess.run(e)

In [12]:
# 创建writer对象
writer = tf.summary.FileWriter('./my_graph', sess.graph)

# cmd-> cd path ->tensorboard --logdir='my_graph'

In [15]:
# 关闭session,writer对象
writer.close()
sess.close()

In [16]:
# 张量
a = tf.constant([[5, 3, 2, 1], [4, 3, 2, 1]], name='input_a')
b = tf.reduce_prod(a, name='prod_b')
c = tf.reduce_sum(a, name='sum_c')
e = c+d

sess = tf.Session()
sess.run(e)

57

In [17]:
sess.run(b)

720